In [1]:
# pip install lightgbm pandas scikit-learn

In [5]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import json
from sklearn.metrics import precision_score, recall_score, fbeta_score, ndcg_score

# df_new = pd.read_csv("INPUT_FILE_PATH")
# train_df, test_df = train_test_split(df_new, test_size=0.2, random_state=42)

# Load the data
train_df1 = pd.read_csv('/kaggle/input/top100-bm25-qld-alqac2024/alqac_task1_final_data/train_2224_final.csv')
train_df2 = pd.read_csv('/kaggle/input/top100-bm25-qld-alqac2024/alqac_task1_final_data/priv_23_final.csv')
train_df3 = pd.read_csv('/kaggle/input/top100-bm25-qld-alqac2024/alqac_task1_final_data/data_qa_final.csv')


train_df15 = pd.read_csv('/kaggle/input/top100-bm25-qld-alqac2024/with mono large/train_2224.csv')
train_df25 = pd.read_csv('/kaggle/input/top100-bm25-qld-alqac2024/with mono large/pri.csv')
train_df35 = pd.read_csv('/kaggle/input/top100-bm25-qld-alqac2024/with mono large/data_qa.csv')


# Concatenate training data
train_df = pd.concat([train_df1, train_df2], ignore_index=True)
train_df = pd.concat([train_df, train_df3], ignore_index=True)

train_df5 = pd.concat([train_df15, train_df25], ignore_index=True)
train_df5 = pd.concat([train_df5, train_df35], ignore_index=True)

# train_df = train_df1

# Load test data
test_df = pd.read_csv('/kaggle/input/top100-bm25-qld-alqac2024/alqac_task1_final_data/pub_24_final.csv')

test_df5 = pd.read_csv('/kaggle/input/top100-bm25-qld-alqac2024/with mono large/pub.csv')

# Display the first few rows of the dataframe
train_df.head()


,question_id,query,law_id,article_id,content,bm25_score,bm25_score_scaled,label,qld_score,bert_score,query_length,content_length,content_length_norm,ViMonoT5_score,ViMonoT5_predict,sbert_contras_score_scaled,sbert_contras_score,article_length
0,01-DS-1,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,"Luật Phòng, chống ma túy",32,Đối tượng bị áp dụng biện pháp xử lý hành chín...,194.345345,1.000000,1,0.873061,1.000000,63,176,0.167264,0.999787,1,1.000000,0.950819,NaN
1,01-DS-1,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,"Luật Phòng, chống ma túy",33,Cai nghiện ma túy cho người từ đủ 12 tuổi đến ...,171.007909,0.849153,0,0.613947,0.999999,63,348,0.372760,0.999222,1,0.382826,0.200552,NaN
2,01-DS-1,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,"Luật Phòng, chống ma túy",40,Quản lý sau cai nghiện ma túy tại nơi cư trú 1...,163.075131,0.797878,0,0.615689,0.732908,63,300,0.315412,0.998248,1,0.282563,0.078668,NaN
3,01-DS-1,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,"Luật Phòng, chống ma túy",55,Quy định chuyển tiếp 1. Kể từ ngày Luật này có...,161.988746,0.790856,0,0.849426,0.006634,63,523,0.581840,0.004146,0,0.591757,0.454539,NaN
4,01-DS-1,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,"Luật Phòng, chống ma túy",35,Cơ sở cai nghiện ma túy công lập 1. Cơ sở cai ...,155.083348,0.746221,0,0.570905,0.999969,63,708,0.802867,0.996210,1,0.395132,0.215511,NaN


In [6]:
print(len(train_df1))
print(len(train_df2))
print(len(train_df3))
print(len(train_df))
print(len(test_df))

38100
11000
71700
120800
20800


In [7]:
print(len(train_df5))
print(len(test_df5))

120800
20800


In [8]:
train_df['ViMonoT5_large_score'] = train_df5['ViMonoT5_score']
test_df['ViMonoT5_large_score'] = test_df5['ViMonoT5_score']

In [9]:
# Add a column for the first two characters of question_id
train_df['question_id_prefix'] = train_df['question_id'].str[:2]
test_df['question_id_prefix'] = test_df['question_id'].str[:2]

# Function to normalize a column within a group
def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

# Group by the first two characters of question_id
grouped = train_df.groupby('question_id_prefix')

train_df['query_length_norm2'] = grouped['query_length'].transform(normalize)
train_df['content_length_norm2'] = grouped['content_length'].transform(normalize)



grouped2 = test_df.groupby('question_id_prefix')

test_df['query_length_norm2'] = grouped2['query_length'].transform(normalize)
test_df['content_length_norm2'] = grouped2['content_length'].transform(normalize)

In [10]:
train_df.head(3)

,question_id,query,law_id,article_id,content,bm25_score,bm25_score_scaled,label,qld_score,bert_score,...,content_length_norm,ViMonoT5_score,ViMonoT5_predict,sbert_contras_score_scaled,sbert_contras_score,article_length,ViMonoT5_large_score,question_id_prefix,query_length_norm2,content_length_norm2
0,01-DS-1,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,"Luật Phòng, chống ma túy",32,Đối tượng bị áp dụng biện pháp xử lý hành chín...,194.345345,1.000000,1,0.873061,1.000000,...,0.167264,0.999787,1,1.000000,0.950819,NaN,0.998801,01,0.323353,0.098341
1,01-DS-1,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,"Luật Phòng, chống ma túy",33,Cai nghiện ma túy cho người từ đủ 12 tuổi đến ...,171.007909,0.849153,0,0.613947,0.999999,...,0.372760,0.999222,1,0.382826,0.200552,NaN,0.997820,01,0.323353,0.204057
2,01-DS-1,Người nghiện ma túy từ đủ 18 tuổi trở lên bị á...,"Luật Phòng, chống ma túy",40,Quản lý sau cai nghiện ma túy tại nơi cư trú 1...,163.075131,0.797878,0,0.615689,0.732908,...,0.315412,0.998248,1,0.282563,0.078668,NaN,0.938885,01,0.323353,0.174554


In [11]:
# Define feature columns and target
features = ['bm25_score_scaled', 
            'qld_score', 
            'bert_score', 
#             'content_length_norm', 
            'ViMonoT5_score', 
#             'query_length', 
            'sbert_contras_score_scaled',
#             'sbert_contras_score',
#             'ratio_qc',
            'query_length_norm2',
            'content_length_norm2',
            'ViMonoT5_large_score'
            ]
path_output = 'ver5'
target = 'label'

# Prepare training and test sets
X_train = train_df[features]
y_train = train_df[target]

X_test = test_df[features]
y_test = test_df[target]

# Group data by 'question_id' for ranking
train_group = train_df.groupby('question_id').size().tolist()
test_group = test_df.groupby('question_id').size().tolist()

# Create LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train, group=train_group)
test_data = lgb.Dataset(X_test, label=y_test, group=test_group, reference=train_data)


In [12]:
import lightgbm as lgb

gbm = lgb.LGBMRanker()

# Create dataset for LightGBM
train_dataset = lgb.Dataset(X_train, label=y_train, group=train_group)

# Define parameters for the ranking model
params = {
    'objective': 'lambdarank',  # Objective for ranking
    'metric': 'ndcg',           # Evaluation metric
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Train the model
gbm = lgb.train(params, train_dataset, num_boost_round=100)

# Save the model if needed
gbm.save_model('lgbm_ranking_model.txt')


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008316 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1973
[LightGBM] [Info] Number of data points in the train set: 120800, number of used features: 8


In [13]:
# Get feature importance
importance = gbm.feature_importance()

# Get feature names
feature_names = X_train.columns

# Combine feature names and importance into a DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
})

# Sort by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print(feature_importance_df)

                      Feature  Importance
7        ViMonoT5_large_score         534
6        content_length_norm2         456
4  sbert_contras_score_scaled         426
2                  bert_score         392
3              ViMonoT5_score         366
1                   qld_score         327
0           bm25_score_scaled         307
5          query_length_norm2         192


In [14]:
# Load the model
loaded_booster = lgb.Booster(model_file='lgbm_ranking_model.txt')

# Feature importance
importance = loaded_booster.feature_importance()
print(f'Feature importance: {importance}')

Feature importance: [307 327 392 366 426 192 456 534]


In [15]:
# Predict on the train data
y_pred_train = gbm.predict(X_train, num_iteration=gbm.best_iteration)

# Add predictions to the train dataframe
train_df['predicted_score'] = y_pred_train

In [16]:
# Predict on the test data
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

# Add predictions to the test dataframe
test_df['predicted_score'] = y_pred

In [17]:
test_df.describe()

,article_id,bm25_score,bm25_score_scaled,label,qld_score,bert_score,content_length,query_length,content_length_norm,ViMonoT5_score,ViMonoT5_predict,sbert_contras_score_scaled,sbert_contras_score,ViMonoT5_large_score,query_length_norm2,content_length_norm2,predicted_score
count,20800.000000,20800.000000,20800.000000,20800.000000,20800.000000,2.080000e+04,20800.000000,20800.000000,20800.000000,2.080000e+04,20800.000000,20800.000000,20800.000000,2.080000e+04,20800.000000,20800.000000,20800.000000
mean,102.522644,41.547062,0.165111,0.010385,0.502479,2.185739e-02,276.863029,44.495192,0.222558,9.667929e-02,0.096394,0.469993,0.196891,4.935289e-02,0.158964,0.166481,-4.429858
std,135.894124,34.230284,0.189977,0.101377,0.183570,1.297671e-01,212.851339,31.614565,0.197215,2.808008e-01,0.295138,0.223777,0.213255,2.043794e-01,0.145689,0.129867,1.126004
min,1.000000,6.163860,0.000000,0.000000,0.000000,0.000000e+00,4.000000,10.000000,0.000000,1.440294e-07,0.000000,0.000000,0.000000,2.174651e-20,0.000000,0.000000,-5.181816
25%,25.000000,21.258159,0.034458,0.000000,0.368143,7.764773e-07,131.000000,25.000000,0.084091,1.533021e-04,0.000000,0.305706,0.000000,3.475157e-05,0.069124,0.077486,-4.990104
50%,50.000000,31.397204,0.096233,0.000000,0.489149,1.831079e-06,219.000000,33.500000,0.164098,9.734239e-04,0.000000,0.462598,0.138406,1.573800e-04,0.108295,0.131178,-4.778514
75%,113.000000,47.631101,0.226597,0.000000,0.625497,8.105251e-06,353.000000,52.000000,0.294615,6.796797e-03,0.000000,0.627698,0.330968,1.386323e-03,0.193548,0.212935,-4.364708
max,689.000000,493.688230,1.000000,1.000000,1.000000,1.000000e+00,1643.000000,227.000000,1.000000,9.999875e-01,1.000000,1.000000,0.993592,9.998769e-01,1.000000,1.000000,4.502680


In [18]:
# Predict on the test data
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

# Add predictions to the test dataframe
test_df['predicted_score'] = y_pred

# Sort data by question_id and predicted_score
test_df_sorted = test_df.sort_values(by=['question_id', 'predicted_score'], ascending=[True, False])

# Calculate NDCG
ndcg_scores = []
for _, group in test_df_sorted.groupby('question_id'):
    ndcg = ndcg_score([group[target].values], [group['predicted_score'].values])
    ndcg_scores.append(ndcg)

# Average NDCG score
average_ndcg = np.mean(ndcg_scores)
print(f'Average NDCG Score: {average_ndcg:.4f}')

# Save the ranked test dataframe to a CSV file
test_df_sorted.to_csv('ranked_test_data.csv', index=False)
print("Ranked test data saved to 'ranked_test_data.csv'")

# Number of unique values in test_df
# unique_values = test_df.nunique()
# print("\nNumber of unique values in each column of test_df:")
# print(unique_values)

Average NDCG Score: 0.9578
Ranked test data saved to 'ranked_test_data.csv'


1 rel

In [19]:
test_df['predicted_label'] = test_df.groupby('question_id')['predicted_score'].transform(lambda x: (x == x.max()).astype(int))

# Calculate precision, recall, and F2 score
precision = precision_score(test_df['label'], test_df['predicted_label'])
recall = recall_score(test_df['label'], test_df['predicted_label'])
f2 = (5 * precision * recall) / (4 * precision + recall)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F2 Score: {f2:.4f}")

Precision: 0.9183
Recall: 0.8843
F2 Score: 0.8909


Thresold

In [13]:
# threshold = 0

# # Apply the threshold to each group's predicted scores
# test_df['predicted_label'] = test_df.groupby('question_id')['predicted_score'].transform(lambda x: (x > threshold).astype(int))

# # Calculate precision, recall, and F2 score
# precision = precision_score(test_df['label'], test_df['predicted_label'])
# recall = recall_score(test_df['label'], test_df['predicted_label'])
# f2 = (5 * precision * recall) / (4 * precision + recall)

# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F2 Score: {f2:.4f}")

In [14]:
# Predict on the test data
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

# Add predictions to the test dataframe
test_df['predicted_score'] = y_pred

# Sort by 'question_id' and 'predicted_score' in descending order
sorted_df = test_df.sort_values(by=['question_id', 'predicted_score'], ascending=[True, False])

# # Group by 'question_id' and take the top 10 entries for each group
# top_10_df = sorted_df.groupby('question_id').head(10).copy()

# # Save the top 10 predictions for each 'question_id' to a CSV file
# top_10_df.to_csv('top_10_pub_test.csv', index=False)

# # Group by 'question_id' and take the top 10 entries for each group
# top_20_df = sorted_df.groupby('question_id').head(20).copy()

# # Save the top 10 predictions for each 'question_id' to a CSV file
# top_20_df.to_csv('top_20_pub_test.csv', index=False)


In [15]:
test_df.head()

,question_id,query,law_id,article_id,content,bm25_score,bm25_score_scaled,label,qld_score,bert_score,content_length,query_length,content_length_norm,ViMonoT5_score,ViMonoT5_predict,sbert_contras_score_scaled,sbert_contras_score,predicted_score,predicted_label
0,03-DS-1,Lưu trú là việc công dân sinh sống tại một địa...,Luật Cư trú,2,"Giải thích từ ngữ Trong Luật này, các từ ngữ d...",108.953765,1.000000,1,1.000000,1.000000e+00,422,50,0.285290,0.996834,1,0.809921,0.875712,3.281252,1
1,03-DS-1,Lưu trú là việc công dân sinh sống tại một địa...,Luật Viên chức,9,Đơn vị sự nghiệp công lập và cơ cấu tổ chức qu...,71.903763,0.498220,0,0.622856,8.772042e-08,304,50,0.197623,0.000085,0,0.422463,0.000000,-4.682321,0
2,03-DS-1,Lưu trú là việc công dân sinh sống tại một địa...,Luật Cư trú,27,Điều kiện đăng ký tạm trú 1. Công dân đến sinh...,69.170868,0.461208,0,0.428003,9.266086e-06,95,50,0.042348,0.010639,0,0.906016,0.000000,-4.363934,0
3,03-DS-1,Lưu trú là việc công dân sinh sống tại một địa...,Hiến pháp,111,1. Chính quyền địa phương được tổ chức ở các đ...,66.474486,0.424690,0,0.610653,3.285884e-07,67,50,0.021545,0.001270,0,0.681792,0.029792,-4.656109,0
4,03-DS-1,Lưu trú là việc công dân sinh sống tại một địa...,Luật Bảo vệ môi trường,123,Phân cấp sự cố môi trường và các giai đoạn ứng...,64.289773,0.395101,0,0.616759,1.104020e-07,266,50,0.169391,0.000241,0,0.294879,0.000000,-4.595174,0


In [16]:
# Assuming your test_df contains the columns: 'question_id', 'law_id', 'article_id', and 'label'
# Filter rows where label is 1
relevant_df = test_df[test_df['predicted_label'] == 1]

# Initialize a list to store the JSON structure
output_list = []

# Group by 'question_id' and aggregate the relevant articles
grouped = relevant_df.groupby('question_id')

for question_id, group in grouped:
    relevant_articles = []
    for _, row in group.iterrows():
        relevant_articles.append({
            "law_id": row['law_id'],
            "article_id": str(row['article_id'])
        })
    output_list.append({
        "question_id": question_id,
        "relevant_articles": relevant_articles
    })

# Save the JSON structure to a file
with open('private_24_1_rel_law.json', 'w', encoding='utf-8') as json_file:
    json.dump(output_list, json_file, ensure_ascii=False, indent=4)

# Print the JSON structure to verify
# print(json.dumps(output_list, ensure_ascii=False, indent=4))


In [17]:
# Load the JSON files
with open('/kaggle/working/private_24_1_rel_law.json', 'r', encoding='utf-8') as f:
    current_data = json.load(f)

with open('/kaggle/input/all-private-test-related-data-alqac-2024/private_test_TASK_1.json', 'r', encoding='utf-8') as f:
    correct_order_data = json.load(f)

# Create a mapping from question_id to index in correct_order_data
order_mapping = {item['question_id']: index for index, item in enumerate(correct_order_data)}

# Sort current_data based on the order in correct_order_data
reordered_data = sorted(current_data, key=lambda x: order_mapping.get(x['question_id'], float('inf')))

# Save the reordered JSON to a new file
with open('private_24_1_rel_law_reordered_file.json', 'w', encoding='utf-8') as f:
    json.dump(reordered_data, f, ensure_ascii=False, indent=4)

In [131]:
top_10_df.head(2)

,question_id,query,law_id,article_id,content,bm25_score,bm25_score_scaled,label,qld_score,bert_score,content_length,query_length,content_length_norm,ViMonoT5_score,ViMonoT5_predict,sbert_contras_score_scaled,predicted_score,predicted_label
0,03-DS-1,Lưu trú là việc công dân sinh sống tại một địa...,Luật Cư trú,2,"Giải thích từ ngữ Trong Luật này, các từ ngữ d...",108.953765,1.00000,1,1.000000,1.000000,422,50,0.285290,0.996834,1,0.809921,4.055128,1
6,03-DS-1,Lưu trú là việc công dân sinh sống tại một địa...,Luật Cư trú,31,Khai báo tạm vắng 1. Công dân có trách nhiệm k...,63.207970,0.38045,0,0.312547,0.000002,687,50,0.482169,0.002909,0,0.762554,-3.710349,0


# Post-processing THUIR

In [20]:
def my_classification_report(df):
    # Calculate precision and recall using sklearn
    precision = precision_score(df['label'], df['predicted_label'])
    recall = recall_score(df['label'], df['predicted_label'])
    
    # Calculate F2 score
    f2 = (5 * precision * recall) / (4 * precision + recall) if (4 * precision + recall) > 0 else 0
    
    return f2, precision, recall

#### Grid-search

In [ ]:
# print(len(top_10_df))
# uwu = top_10_df[0:50]
# print(len(uwu))

In [ ]:
# grouped_uwu = uwu.groupby('question_id')
# print(len(grouped_uwu))

In [ ]:
# df = top_10_df
# df = top_10_train_df
df = test_df
import time
start_time = time.time()

# Assuming df is your DataFrame and it is sorted by question and then by predicted_score
def grid_search(df):
    rs = []
    for percent in range(10, 100, 2):
        p = percent / 100
        for h in range(2, 5):
            for l in range(1, h):
                df_copy = df.copy()
                new_predictions = []

                for qid, group in df.groupby('question_id'):
                    # Initialize the new answer list
                    now_answer = group.iloc[:l].copy()
                    biggest = group['predicted_score'].iloc[0]
#                     print(biggest)

                    for i in range(l, min(h, len(group))):
                        if biggest > 0:
                            if group['predicted_score'].iloc[i] >= p * biggest:
                                now_answer = pd.concat([now_answer, group.iloc[[i]]])
                            else:
                                break
                        elif group['predicted_score'].iloc[i] >= biggest / p:
                            now_answer = pd.concat([now_answer, group.iloc[[i]]])
                        else:
                            break

                    new_predictions.append(now_answer)

                # Update the predicted labels based on new_predictions
                for prediction in new_predictions:
                    df_copy.loc[prediction.index, 'predicted_label'] = 1

                # Calculate performance using custom function
                f2_score, precision, recall = my_classification_report(df_copy)
                
                rs.append(((p, h, l), f2_score))

    return rs

# Example usage:
# Assuming your DataFrame is stored in df
results = grid_search(df)

# To get the best performing parameters based on f2_score
best_params = max(results, key=lambda x: x[1])
print(f"Best Parameters: p={best_params[0][0]}, h={best_params[0][1]}, l={best_params[0][2]}")
print(f"Best F2 Score: {best_params[1]}")
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Use the best parameters to generate final predictions
def apply_best_params(df, p, h, l):
    df_copy = df.copy()
    new_predictions = []

    for qid, group in df.groupby('question_id'):
        # Initialize the new answer list
        now_answer = group.iloc[:l].copy()
        biggest = group['predicted_score'].iloc[0]

        for i in range(l, min(h, len(group))):
            if biggest > 0:
                if group['predicted_score'].iloc[i] >= p * biggest:
                    now_answer = pd.concat([now_answer, group.iloc[[i]]])
                else:
                    break
            elif group['predicted_score'].iloc[i] >= biggest / p:
                now_answer = pd.concat([now_answer, group.iloc[[i]]])
            else:
                break

        new_predictions.append(now_answer)

    # Update the predicted labels based on new_predictions
    for prediction in new_predictions:
        df_copy.loc[prediction.index, 'predicted_label'] = 1

    return df_copy

In [ ]:
# Apply the best parameters to get the final DataFrame
best_p, best_h, best_l = best_params[0]
print(best_params[0])
# result_final_df = apply_best_params(top_10_df, best_p, best_h, best_l)
result_final_df = apply_best_params(test_df, best_p, best_h, best_l)

In [ ]:
precision = precision_score(result_final_df['label'], result_final_df['predicted_label'])
recall = recall_score(result_final_df['label'], result_final_df['predicted_label'])
f2 = (5 * precision * recall) / (4 * precision + recall)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F2 Score: {f2:.4f}")

In [ ]:
# Save the top 10 predictions for each 'question_id' to a CSV file
result_final_df.to_csv(path_output+'updated.csv', index=False)

In [ ]:
# # Filter rows where label is 1
# relevant_df = result_final_df[result_final_df['predicted_label'] == 1]

# # Initialize a list to store the JSON structure
# output_list = []

# # Group by 'question_id' and aggregate the relevant articles
# grouped = relevant_df.groupby('question_id')

# for question_id, group in grouped:
#     relevant_articles = []
#     for _, row in group.iterrows():
#         relevant_articles.append({
#             "law_id": row['law_id'],
#             "article_id": str(row['article_id'])
#         })
#     output_list.append({
#         "question_id": question_id,
#         "relevant_articles": relevant_articles
#     })

# # Save the JSON structure to a file
# with open(path_output+'grid_search_updated.json', 'w', encoding='utf-8') as json_file:
#     json.dump(output_list, json_file, ensure_ascii=False, indent=4)

## Post-processing for multiple rel_law

In [ ]:
# def update_labels(add_suffixdf):
#     # Cần tune đống này
#     range_between_1st_and_2nd_pred_score = 1
    
    
#     grouped = df.groupby('question_id')
    
#     for name, group in grouped:
#         if len(group) > 1:  # Ensure there are at least two rows
#             first_row = group.iloc[0]
#             second_row = group.iloc[1]

#             # Check conditions
#             if second_row['predicted_score'] > 0 and abs(first_row['predicted_score'] - second_row['predicted_score']) < range_between_1st_and_2nd_pred_score:
#                 # Update label of the second row to 1
#                 df.loc[second_row.name, 'predicted_label'] = 1
    
#     return df

In [ ]:
# updated_top_10_df = update_labels(top_10_df)

In [ ]:
# # Calculate precision, recall, and F2 score
# precision = precision_score(updated_top_10_df['label'], updated_top_10_df['predicted_label'])
# recall = recall_score(updated_top_10_df['label'], updated_top_10_df['predicted_label'])
# f2 = (5 * precision * recall) / (4 * precision + recall)

# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F2 Score: {f2:.4f}")

In [ ]:
# # Save the top 10 predictions for each 'question_id' to a CSV file
# updated_top_10_df.to_csv(path_output+'updated.csv', index=False)

In [ ]:
# # Filter rows where label is 1
# relevant_df = updated_top_10_df[updated_top_10_df['predicted_label'] == 1]

# # Initialize a list to store the JSON structure
# output_list = []

# # Group by 'question_id' and aggregate the relevant articles
# grouped = relevant_df.groupby('question_id')

# for question_id, group in grouped:
#     relevant_articles = []
#     for _, row in group.iterrows():
#         relevant_articles.append({
#             "law_id": row['law_id'],
#             "article_id": str(row['article_id'])
#         })
#     output_list.append({
#         "question_id": question_id,
#         "relevant_articles": relevant_articles
#     })

# # Save the JSON structure to a file
# with open(path_output+'updated.json', 'w', encoding='utf-8') as json_file:
#     json.dump(output_list, json_file, ensure_ascii=False, indent=4)

### Grid search - bach

In [21]:
# Predict on the test data
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

# Add predictions to the test dataframe
test_df['predicted_score'] = y_pred

# Sort by 'question_id' and 'predicted_score' in descending order
sorted_df = test_df.sort_values(by=['question_id', 'predicted_score'], ascending=[True, False])

# Group by 'question_id' and take the top 10 entries for each group
top_10_df = sorted_df.groupby('question_id').head(10).copy()

In [22]:
def apply_relu(df, column_name='predicted_score'):
    df.loc[df[column_name] < 0, column_name] = 0
    return df

# Apply relu
train_df = apply_relu(train_df, 'predicted_score')
test_df = apply_relu(test_df, 'predicted_score')

In [23]:
# # Define the softmax function
# def softmax(x):
#     e_x = np.exp(x - np.max(x))
#     return e_x / e_x.sum(axis=0)

# # Apply softmax scaling within each question_id group
# test_df['predicted_score_scaled'] = test_df.groupby('question_id')['predicted_score'].transform(softmax)
# test_df.head()

In [24]:
# top_10_df['predicted_score_scaled'] = top_10_df.groupby('question_id')['predicted_score'].transform(softmax)

In [25]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score

def grid_search_f2(df, P_range, max_articles_range):
    best_f2 = -1
    best_params = None

    # Iterate over P values
    for P in P_range:
        # Iterate over max number of relevant articles
        for max_articles in max_articles_range:
            df['predicted_label'] = 0  # Initialize all predicted labels to 0
            
            # Group by question_id and apply the threshold and max articles constraint
            for question_id, group in df.groupby('question_id'):
                S = group['predicted_score'].max()
                threshold = P * S
                
                # Filter articles exceeding the threshold
                relevant_articles = group[group['predicted_score'] >= threshold]
                
                # Limit to top max_articles if necessary
                if len(relevant_articles) > max_articles:
                    relevant_articles = relevant_articles.nlargest(max_articles, 'predicted_score')
                
                df.loc[relevant_articles.index, 'predicted_label'] = 1
            
            # Calculate F2 score
            f2, precision, recall = my_classification_report(df)
#             precision = precision_score(df['label'], df['predicted_label'])
#             recall = recall_score(df['label'], df['predicted_label'])
#             f2 = (5 * precision * recall) / (4 * precision + recall)
            
            # Update best parameters if current F2 score is better
            if f2 > best_f2:
                best_f2 = f2
                best_params = (P, max_articles)
    
    return best_params, best_f2


# Define ranges for P and max number of relevant articles
P_range = np.arange(0, 1.01, 0.01)
max_articles_range = range(1,5)

# Perform grid search
best_params, best_f2 = grid_search_f2(test_df, P_range, max_articles_range)

print("Best Parameters (P, max_articles):", best_params)
print("Best F2 Score:", best_f2)

Best Parameters (P, max_articles): (0.84, 2)
Best F2 Score: 0.88724584103512


In [26]:
def apply_best_params_1(df, best_params):
    P, max_articles = best_params
    df['predicted_label'] = 0  # Initialize all predicted labels to 0
    
    # Group by question_id and apply the threshold and max articles constraint
    for question_id, group in df.groupby('question_id'):
        S = group['predicted_score'].max()
        threshold = P * S
        
        # Filter articles exceeding the threshold
        relevant_articles = group[group['predicted_score'] >= threshold]
        
        # Limit to top max_articles if necessary
        if len(relevant_articles) > max_articles:
            relevant_articles = relevant_articles.nlargest(max_articles, 'predicted_score')
        
        df.loc[relevant_articles.index, 'predicted_label'] = 1
    
    return df

In [24]:
# Apply the best parameters to the test dataset
top_10_df = apply_best_params_1(top_10_df, best_params)

# Calculate F1 score for the test set
top_10_df_f2,_,__ = my_classification_report(top_10_df)
print("Top 10 Test F2 Score:", top_10_df_f2)

Top 10 Test F2 Score: 0.8661778185151239


In [27]:
# Apply the best parameters to the test dataset
test_df_2 = apply_best_params_1(test_df, best_params)

# Calculate F1 score for the test set
test_f2,_,__ = my_classification_report(test_df_2)
print("Test precision score:", _)
print("Test recall score:", __)
print("Test F2 Score:", test_f2)

Test precision score: 0.8807339449541285
Test recall score: 0.8888888888888888
Test F2 Score: 0.88724584103512


# Private test 

In [25]:
priv_24 = pd.read_csv('/kaggle/input/all-private-test-related-data-alqac-2024/bm25_bert_len_output_data_private_24_vimonot5_sbert.csv')
priv_245 = pd.read_csv('/kaggle/input/all-private-test-related-data-alqac-2024/private_2024.csv') 
priv_24['ViMonoT5_large_score'] = priv_245['ViMonoT5_score']

In [26]:
# Add a column for the first two characters of question_id
priv_24['question_id_prefix'] = priv_24['question_id'].str[:2]


# Group by the first two characters of question_id
grouped3 = priv_24.groupby('question_id_prefix')

priv_24['query_length_norm2'] = grouped3['query_length'].transform(normalize)
priv_24['content_length_norm2'] = grouped3['content_length'].transform(normalize)


In [27]:
X_priv = priv_24[features]

# Predict on the test data
y_priv_pred = gbm.predict(X_priv, num_iteration=gbm.best_iteration)

# Add predictions to the test dataframe
priv_24['predicted_score'] = y_priv_pred

# Sort by 'question_id' and 'predicted_score' in descending order
sorted_df = priv_24.sort_values(by=['question_id', 'predicted_score'], ascending=[True, False])

# Group by 'question_id' and take the top 10 entries for each group
top_20_df = sorted_df.groupby('question_id').head(20).copy()
top_20_df.to_csv('private_test_24_top_20.csv', index=False)

priv_24.to_csv('private_test_24_no_relu.csv', index=False)

priv_24_2 = apply_relu(priv_24, 'predicted_score')
priv_24_2.head()

,question_id,query,law_id,article_id,content,bm25_score,bm25_score_scaled,label,query_length,bert_score,...,qld_score,content_length,content_length_norm,sbert_contras_score,sbert_contras_score_scaled,ViMonoT5_large_score,question_id_prefix,query_length_norm2,content_length_norm2,predicted_score
0,04-DS-1,"Trong nguyên tắc về quản lý viên chức, việc tu...",Luật Viên chức,6,Các nguyên tắc quản lý viên chức 1. Bảo đảm sự...,91.598930,1.000000,0,47,1.000000,...,0.529409,166,0.082609,0.713399,1.000000,0.999710,04,0.211765,0.092194,3.243556
1,04-DS-1,"Trong nguyên tắc về quản lý viên chức, việc tu...",Luật Viên chức,7,Vị trí việc làm 1. Vị trí việc làm là công việ...,70.130809,0.650504,0,47,0.000025,...,0.867549,98,0.040373,0.537091,0.821012,0.128762,04,0.211765,0.050400,0.000000
2,04-DS-1,"Trong nguyên tắc về quản lý viên chức, việc tu...",Luật Viên chức,31,"Bổ nhiệm, thay đổi chức danh nghề nghiệp 1. Vi...",66.734567,0.595214,0,47,0.000005,...,0.658135,241,0.129193,-0.011087,0.264504,0.004936,04,0.211765,0.138291,0.000000
3,04-DS-1,"Trong nguyên tắc về quản lý viên chức, việc tu...",Luật Viên chức,37,Bổ nhiệm viên chức quản lý 1. Việc bổ nhiệm vi...,64.113004,0.552536,0,47,0.000004,...,0.791527,265,0.144099,-0.044975,0.230101,0.000396,04,0.211765,0.153042,0.000000
4,04-DS-1,"Trong nguyên tắc về quản lý viên chức, việc tu...",Luật Viên chức,32,Thay đổi vị trí việc làm 1. Khi đơn vị sự nghi...,63.772650,0.546995,0,47,0.000007,...,0.642102,143,0.068323,-0.079565,0.194985,0.002454,04,0.211765,0.078058,0.000000


In [28]:
priv_24_3 = apply_best_params_1(priv_24_2, best_params)

In [29]:
priv_24_3.to_csv('private_test_24_th355.csv', index=False)

In [30]:
type(priv_24_3)

pandas.core.frame.DataFrame

In [31]:
priv_24_3.head(1)

,question_id,query,law_id,article_id,content,bm25_score,bm25_score_scaled,label,query_length,bert_score,...,content_length,content_length_norm,sbert_contras_score,sbert_contras_score_scaled,ViMonoT5_large_score,question_id_prefix,query_length_norm2,content_length_norm2,predicted_score,predicted_label
0,04-DS-1,"Trong nguyên tắc về quản lý viên chức, việc tu...",Luật Viên chức,6,Các nguyên tắc quản lý viên chức 1. Bảo đảm sự...,91.59893,1.0,0,47,1.0,...,166,0.082609,0.713399,1.0,0.99971,04,0.211765,0.092194,3.243556,1


In [32]:
# Filter rows where label is 1
relevant_df = priv_24_3[priv_24_3['predicted_label'] == 1]

# Initialize a list to store the JSON structure
output_list = []

# Group by 'question_id' and aggregate the relevant articles
grouped = relevant_df.groupby('question_id')

for question_id, group in grouped:
    relevant_articles = []
    for _, row in group.iterrows():
        relevant_articles.append({
            "law_id": row['law_id'],
            "article_id": str(row['article_id'])
        })
    output_list.append({
        "question_id": question_id,
        "relevant_articles": relevant_articles
    })

# Save the JSON structure to a file
with open('private_test_24_th355.json', 'w', encoding='utf-8') as json_file:
    json.dump(output_list, json_file, ensure_ascii=False, indent=4)

In [2]:
# Load the JSON files
import json 

with open('/kaggle/input/output-a-trung/label.json', 'r', encoding='utf-8') as f:
    current_data = json.load(f)

with open('/kaggle/input/all-private-test-related-data-alqac-2024/private_test_TASK_1.json', 'r', encoding='utf-8') as f:
    correct_order_data = json.load(f)

# Create a mapping from question_id to index in correct_order_data
order_mapping = {item['question_id']: index for index, item in enumerate(correct_order_data)}

# Sort current_data based on the order in correct_order_data
reordered_data = sorted(current_data, key=lambda x: order_mapping.get(x['question_id'], float('inf')))

# Save the reordered JSON to a new file
with open('nowj_run3_anh_trung.json', 'w', encoding='utf-8') as f:
    json.dump(reordered_data, f, ensure_ascii=False, indent=4)

# To json

In [82]:
# Save the top 10 predictions for each 'question_id' to a CSV file
test_df_2.to_csv(path_output+'updated.csv', index=False)

# Filter rows where label is 1
relevant_df = test_df_2[test_df_2['predicted_label'] == 1]

# Initialize a list to store the JSON structure
output_list = []

# Group by 'question_id' and aggregate the relevant articles
grouped = relevant_df.groupby('question_id')

for question_id, group in grouped:
    relevant_articles = []
    for _, row in group.iterrows():
        relevant_articles.append({
            "law_id": row['law_id'],
            "article_id": str(row['article_id'])
        })
    output_list.append({
        "question_id": question_id,
        "relevant_articles": relevant_articles
    })

# Save the JSON structure to a file
with open(path_output+'updated.json', 'w', encoding='utf-8') as json_file:
    json.dump(output_list, json_file, ensure_ascii=False, indent=4)

In [85]:
data_correct_format_path = '/kaggle/input/pub-test-no-label/public_test.json'


with open(data_correct_format_path, 'r', encoding='utf-8') as file:
    data1 = json.load(file)


data2 = output_list

# Matching data1 with data2 to get the correct format
def match_data(data1, data2):
    data2_dict = {item["question_id"]: item["relevant_articles"] for item in data2}
    matched_data = []

    for item in data1:
        question_id = item["question_id"]
        relevant_articles = data2_dict.get(question_id, [])
        matched_data.append({
            "question_id": question_id,
            "relevant_articles": relevant_articles
        })
    
    return matched_data

# Get the matched data
matched_data = match_data(data1, data2)

# Print the matched data
import json
with open(path_output+'updated_reordered.json', 'w', encoding='utf-8') as json_file:
    json.dump(matched_data, json_file, ensure_ascii=False, indent=4)

### Grid search hướng cũ của Duy

In [70]:
def update_labels(df):
    # Cần tune đống này
    range_between_1st_and_2nd_pred_score = 1
    
    
    grouped = df.groupby('question_id')
    
    for name, group in grouped:
        if len(group) > 1:  # Ensure there are at least two rows
            first_row = group.iloc[0]
            second_row = group.iloc[1]

            # Check conditions
            if second_row['predicted_score'] > 0 and abs(first_row['predicted_score'] - second_row['predicted_score']) < range_between_1st_and_2nd_pred_score:
                # Update label of the second row to 1
                df.loc[second_row.name, 'predicted_label'] = 1
    
    return df

In [71]:
updated_df = update_labels(test_df)
# Calculate F1 score for the test set
test_f2,_,__ = my_classification_report(updated_df)
print("Test precision score:", _)
print("Test recall score:", __)
print("Test F2 Score:", test_f2)

Test precision score: 0.825531914893617
Test recall score: 0.8981481481481481
Test F2 Score: 0.8826205641492265
